In [1]:
from __future__ import print_function
import argparse,random
from math import log10
%pylab
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from utils.data import get_training_set, get_test_set
from torch.nn.modules.module import _addindent

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['log10', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# model 1을 사용할시에
from net.model import Net
# model 2을 사용할시에
#from net.model_dw import Net

In [3]:
# PSNR 값을 구한다.
def PSNR(net):
    avg_psnr = 0
    for batch in testing_data_loader:
        input, target = Variable(batch[0]), Variable(batch[1])
        prediction = net(input)
        mse = criterion(prediction, target)
        psnr = 10 * log10(1 / mse.data[0])
        avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))

In [4]:
# cuda import
cuda = True
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")
torch.manual_seed(random.randint(1,1000))
if cuda:
    torch.cuda.manual_seed(random.randint(1,1000))

In [5]:
# dataset import
train_set = get_training_set(2,"BSDS300")
test_set = get_test_set(2,"BSDS300")
training_data_loader = DataLoader(dataset=train_set, num_workers=6, batch_size=16, shuffle=True)
testing_data_loader = DataLoader(dataset=test_set, num_workers=6, batch_size=100, shuffle=False)

In [6]:
# load weigh1
model=torch.load('weight1')
# load weigh2
#model=torch.load('weight2')
keys=model.keys()

# Architecture
___

```
self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
self.conv2=nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
self.conv3=nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
```

<br>
<br>
__upscale_factor는 2 이다__

# Pruning method
___

1. Pruning을 할 수 있는 spot을 선택 
1. 1개를 선택하여 prunging 한 후에 Net의 아키텍쳐를 바꾼다.
1. L1 norm 을 기준으로 pruning 한다.
1. PSNR 값을 구한다.
1. retraining을 진행.
1. 반복(일정 PSNR 이하 로 내려가기전까지)

In [7]:
alpla=64
beta=64
gamma=32
pruning=0
train=Net(2)
upscale_factor=2

In [8]:
def modify(net):
    global alpla
    global gamma
    global beta
    net.conv1 = nn.Conv2d(1, alpla, (5, 5), (1, 1), (2, 2))
    net.conv2=nn.Conv2d(alpla, beta, (3, 3), (1, 1), (1, 1))
    net.conv3=nn.Conv2d(beta, gamma, (3, 3), (1, 1), (1, 1))
    net.conv4 = nn.Conv2d(gamma, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))

def PSNR(net):
    criterion = nn.MSELoss()
    avg_psnr = 0
    for batch in testing_data_loader:
        input, target = Variable(batch[0]), Variable(batch[1])
        prediction = net(input)
        mse = criterion(prediction, target)
        psnr = 10 * log10(1 / mse.data[0])
        avg_psnr += psnr
    return avg_psnr/len(testing_data_loader)    

In [ ]:
keys_list=list(keys)
#print(keys_list)
# ['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias', 'conv4.weight', 'conv4.bias']
psnr_list=[]
pruning+=1
for i in range(0,len(keys_list)-1,2):
    
    if i is 0:
        alpla-=pruning
        modify(train)
    elif i is 2:
        beta-=pruning
        modify(train)
    elif i is 4:
        gamma-=pruning
        modify(train)
        pruning+=1
    for j in range(len(model[keys_list[i]])):
        
        model=torch.load('weight1')
        weight_matrix=model[keys_list[i]]
        bias_matrix=model[keys_list[i+1]]
        if j is 0:
            #print(keys_list[i],keys_list[i+1])
            model[keys_list[i]]=weight_matrix[1:len(model[keys_list[i]])]
            model[keys_list[i+1]]=bias_matrix[1:len(model[keys_list[i]])+1]        
        elif j is len(model[keys_list[i]])-1:
            model[keys_list[i]]=weight_matrix[0:len(model[keys_list[i]])-1]                    
            model[keys_list[i+1]]=bias_matrix[0:len(model[keys_list[i]])]
        else:
            model[keys_list[i]]=torch.cat((weight_matrix[0:j],weight_matrix[j+1:len(model[keys_list[i]])]))
            model[keys_list[i+1]]=torch.cat((bias_matrix[0:j],bias_matrix[j+1:len(model[keys_list[i]])+1]))
        if i is 0:
            model[keys_list[i+2]].resize_(alpla,beta,3,3)
        elif i is 2:
            model[keys_list[i+2]].resize_(beta,gamma,3,3)
        elif i is 4:
            model[keys_list[i+2]].resize_(gamma,upscale_factor ** 2,3,3)
        train.load_state_dict(model)
        prnr=PSNR(train)
        print('conv',i,'num',j,'psnr',prnr)
        psnr_list.append(tuple([i,j,prnr]))

/usr/local/lib/python3.5/dist-packages/torch/nn/modules/module.py:514: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  own_state[name].copy_(param)


conv 0 num 0 psnr 9.734756387825515
conv 0 num 1 psnr 9.406958125684103
conv 0 num 2 psnr 9.243455553136652


In [ ]:
def pruning(net):
    pass

In [ ]:
def calculate(net):
    pass

In [ ]:
def retrain(net):
    pass

In [ ]:
def L2_distance(net):
    pass